In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

In [34]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\alexl\.wdm\drivers\chromedriver\win32\98.0.4758.102]


### Visit the NASA Mars news page

In [3]:
url = 'https://redplanetscience.com'
browser.visit(url)

# Add 1s delay and find class 'list_text'
browser.is_element_present_by_css('div.list_text', wait_time= 1)

True

In [4]:
news_soup = soup(browser.html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [5]:
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title 

'7 Things to Know About the Mars 2020 Perseverance Rover Mission'

In [6]:
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"NASA's next rover to the Red Planet is slated to launch no earlier than July 30. These highlights will get you up to speed on the ambitious mission."

### Featured Images

In [7]:
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [8]:
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [9]:
img_soup = soup(browser.html, 'html.parser')

In [10]:
img_url_rel = img_soup.find('img', class_= 'fancybox-image').get('src')

img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

### Mars Facts

In [11]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns = ['description', 'Mars', 'Earth']
df.set_index('description', inplace= True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [12]:
df.to_html().split()

['<table',
 'border="1"',
 'class="dataframe">',
 '<thead>',
 '<tr',
 'style="text-align:',
 'right;">',
 '<th></th>',
 '<th>Mars</th>',
 '<th>Earth</th>',
 '</tr>',
 '<tr>',
 '<th>description</th>',
 '<th></th>',
 '<th></th>',
 '</tr>',
 '</thead>',
 '<tbody>',
 '<tr>',
 '<th>Mars',
 '-',
 'Earth',
 'Comparison</th>',
 '<td>Mars</td>',
 '<td>Earth</td>',
 '</tr>',
 '<tr>',
 '<th>Diameter:</th>',
 '<td>6,779',
 'km</td>',
 '<td>12,742',
 'km</td>',
 '</tr>',
 '<tr>',
 '<th>Mass:</th>',
 '<td>6.39',
 '×',
 '10^23',
 'kg</td>',
 '<td>5.97',
 '×',
 '10^24',
 'kg</td>',
 '</tr>',
 '<tr>',
 '<th>Moons:</th>',
 '<td>2</td>',
 '<td>1</td>',
 '</tr>',
 '<tr>',
 '<th>Distance',
 'from',
 'Sun:</th>',
 '<td>227,943,824',
 'km</td>',
 '<td>149,598,262',
 'km</td>',
 '</tr>',
 '<tr>',
 '<th>Length',
 'of',
 'Year:</th>',
 '<td>687',
 'Earth',
 'days</td>',
 '<td>365.24',
 'days</td>',
 '</tr>',
 '<tr>',
 '<th>Temperature:</th>',
 '<td>-87',
 'to',
 '-5',
 '°C</td>',
 '<td>-88',
 'to',
 '58°C</td>'

## Scrape Mars Hemisphere Images and Titles

In [46]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [47]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
hem_soup = soup(browser.html, 'html.parser')
hem_soup = hem_soup.find('div', class_='collapsible results')
hem_soup = hem_soup.find_all('div', class_='item')

for item in hem_soup:
    title = item.find('h3').get_text()

    browser.links.find_by_partial_text(item.find('h3').get_text()).click()
    browser.links.find_by_partial_text('Sample').click()
    img = item.find('img').get('src')

    hemisphere_image_urls.append({'img_url':img, 'title': title})

    browser.back()


In [48]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': '/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': '/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': '/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': '/cache/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [16]:
browser.quit()